In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

# 修正したモデルから卒論時に集計したデータを作成する

1. 表

| ベンチマーク名 | 平均誤差率(%) | コスト比(%) |
|---------|----------|---------|


2. 表

| ベンチマーク名 | 採用割合(最大MAPE(%), 最小MAPE(%)) |
|---------|----------------------------|
|         | モデル(1), モデル(2), ...        |


* 平均誤差率：大規模実行時の関数コール回数との比較
* MAPE：トレーニングデータとの比較

In [ ]:
plt.figure(figsize=(4, 3))
plt.xlabel("使用したプロファイル数")
plt.ylabel("平均誤差率(%)")

In [ ]:
# pd.get_option("display.max_columns")
# pd.get_option("display.max_rows")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [ ]:
# ベンチマーク名・関数名・プロセス数・問題サイズを指定することで、その条件での関数コール回数を取得する関数

def returnSpecificData(benchmarkName="cg", functionName=".TAU_application", process=256, benchmarkClass="D"):
    targetRawDF = returnRawDF(Benchmark=benchmarkName, functionName=functionName, benchmarkClass=[
                              benchmarkClass], FixedProcess=process, Processes=[process], FixedBenchmarkClass=benchmarkClass)
    return targetRawDF.iat[0, 0]
# returnSpecificData(benchmarkName="mg", functionName="BUBBLE", process=256, benchmarkClass="B")

In [ ]:
# benchmarksからbt, spを除外する
benchmarks = [benchmark for benchmark in benchmarks if benchmark !=
              'bt' and benchmark != 'sp']
# pandasのDFをprintした時の幅を広げる
pd.set_option('display.width', 100)

In [ ]:
dictTmp = returnDictForPlotPerNumOfUsedData(Benchmark=benchmarks, fix="Class", benchmarkClass=[
    "A", "B", "C", "D"], FixedProcess=64, Processes=[1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchmarkClass="C")

In [ ]:
pd.options.display.float_format = '{:.4g}'.format

tmpDF = pd.DataFrame()
for benchmark in benchmarks:
    listToLearn = [1, 2, 4, 8, 16, 32, 64, 128]
    listToPredict = [256]
    benchmark_x = dictTmp[benchmark]["x"]
    benchmark_y = dictTmp[benchmark]["y"]
    index = benchmark_x.index(len(listToLearn))
    MAPE = benchmark_y[index]
    relativeCost = returnRelativeCost(benchmark=benchmark, variablesToLearn=listToLearn,
                                      variablesToPredict=listToPredict, fixedClassOrProcess="Class", fixed="C")
    dictRowData = {"ベンチマーク名": benchmark.upper(
    ), "平均絶対誤差率[％]": MAPE, "相対コスト[％]": relativeCost}
    iDF = pd.DataFrame.from_dict(dictRowData, orient='index').T
    tmpDF = tmpDF.append(iDF)
tmpDFMean = tmpDF.mean()
type(tmpDFMean)
print(tmpDF.to_latex(index=False))

In [ ]:
# dictTmp

plt.figure(figsize=(5.72, 4), dpi=200)
for benchmark in list(dictTmp.keys()):
    x = dictTmp[benchmark]["x"]
    y = dictTmp[benchmark]["y"]
    plt.plot(x, y, marker='o', label=benchmark.upper())
    plt.legend()
    plt.xlabel("使用したプロファイル数")
    plt.ylabel("平均絶対誤差率[％]")

In [ ]:
plt.figure(figsize=(5.72, 4), dpi=200)

# Extra-PでfixProcessデータを入力して出力したモデルの図時
plot_x = np.linspace(0.8, 256, 500)
# -3590464.6990329633 + 3759195.349891038 * p^(1/4)
plot_y = []
for x in plot_x:
    plot_y.append(2286768.3333333326 + 301997.61904761934 * math.log2(x)**(1))
plt.plot(plot_x, plot_y, label="ExtraP")

x = [1, 2, 4, 8, 16, 32, 64, 128]
y = [1984770.0, 2263540.0, 2821070.0, 3936140.0,
     3936140.0, 3936140.0, 3936140.0, 3936140.0]
x = np.array(x).reshape(-1, 1)
y = np.array(y).reshape(-1, 1)
plt.scatter(x, y, marker="o", label="予測に用いた関数コール回数")
plot_x = np.array(plot_x).reshape(-1, 1)
x_target = [256]
y_target = [3936140]
plt.scatter(x_target, y_target, marker="o", label="予測したい関数コール回数の実測値")

benchmarkName = "CG"
functionName = "ICNVRT"

# 線形モデル
# 対数モデル

# 反比例モデル
modelIpMk2 = ModelIp_mk2(train_x=x, train_y=y, target_x=x_target, target_y=y_target,
                         benchmark_name=benchmarkName, function_name=functionName)
modelIpMk2.calc_lr()
plot_y_IpMk2 = modelIpMk2.predict(plot_x)
plt.plot(plot_x, plot_y_IpMk2, label="反比例モデル")
# 線形飽和モデル
modelBranchMk2 = ModelBranch_mk2(train_x=x, train_y=y, target_x=x_target,
                                 target_y=y_target, benchmark_name=benchmarkName, function_name=functionName)
modelBranchMk2.calc_lr()
plot_y_BranchMk2 = modelBranchMk2.predict(plot_x)
plt.plot(plot_x, plot_y_BranchMk2, label="線形飽和モデル")
# # 線形モデル
# model_lin = ModelLin(x, y, "CG", "ICNVRT", test_ratio=0)
# model_lin.calc_lr()
# plot_y_lin = model_lin.predict(plot_x)
# plt.plot(plot_x, plot_y_lin, label="線形モデル")
# # 対数モデル
# model_log10 = ModelLog10(x, y, "CG", "ICNVRT", test_ratio=0)
# model_log10.calc_lr()
# plot_y_log10 = model_log10.predict(plot_x)
# plt.plot(plot_x, plot_y_log10, label="対数モデル")
# # 反比例モデル
# model_ip = ModelIP(x, y, "CG", "ICNVRT", test_ratio=0)
# model_ip.calc_lr()
# plot_y_ip = model_ip.predict(plot_x)
# plt.plot(plot_x, plot_y_ip, label="反比例モデル")
# # 線形飽和モデル
# model_branch = ModelBranch(x, y, "CG", "ICNVRT", test_ratio=0)
# model_branch.calc_lr()
# plot_y_branch = model_branch.predict(plot_x)
# plt.plot(plot_x, plot_y_branch, label="線形飽和モデル")
# 凡例の表示
plt.legend()
# 軸ラベルの設定
plt.ylabel("関数コール回数")
plt.xlabel("実行コア数")

plt.scatter(x, y, marker="o")

In [ ]:
# 実際にプロットする


# print(f"fix={fix}, benchmarkClasses={benchmarkClasses}, fixedProcess={fixedProcess}, Processes={processes}, FixedBenchmarkClass={fixedBenchmarkClass}")
# print(f"targetNumOfProcess={targetNumOfProcess}, targetProblemSize={fixedBenchmarkClass}, fix={fix}")

# DF = returnRawDFperBenchmark(Benchmark="mg", fix="Process", benchmarkClass=["A", "B", "C", "D"], Processes=[
#                              1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchmarkClass="B", FixedProcess=64)
# DF.dropna(how='any')
# DF

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

In [ ]:
benchmarkNamesExcludeBTSP = ["cg", "ep", "ft", "is", "lu", "mg"]
# classes = ["A", "B", "C", "D"]
classes = ["B"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]
targetIndex = -1
csvDirPath = "./csv_files/"

dfByDatumExcludeBTSP = returnDFSummarizedData(
    benchmarkNames=benchmarkNamesExcludeBTSP, classes=classes, processes=processes, targetIndex=targetIndex, csvDirPath=csvDirPath)
# dfByDatumExcludeBTSP

dictForLatexTable = {}
numOfData = 0
for benchmarkName in benchmarkNamesExcludeBTSP:
    dictForLatexTable[benchmarkName] = dfByDatumExcludeBTSP[dfByDatumExcludeBTSP["benchmarkName"] == benchmarkName]
    numOfData += len(
        dfByDatumExcludeBTSP[dfByDatumExcludeBTSP["benchmarkName"] == benchmarkName])

numOfData

In [ ]:


listForDF = []

for benchmarkName in benchmarkNamesExcludeBTSP:
    listForDF.append(returnSeriesOfDatumPerBenchmark(
        inputDF=dictForLatexTable[benchmarkName]))
DF = pd.DataFrame(listForDF)
print(DF.to_latex(index=False))

In [ ]:
test_returnSeriesOfDatumPerBenchmark()

In [ ]:
resultIs = dictForLatexTable["is"]
# resultIs
resultIsAtModelBranch = resultIs[resultIs["objectBestModelName"]
                                 == "ModelBranch"]
datumX = resultIsAtModelBranch["usedDataX"].tolist()
datumY = resultIsAtModelBranch["usedDataY"].tolist()

datumX
datumY

# returnSeriesOfData(benchmarkName="is", functionName="double_randlc(double_*_double_*)", rawX=dataX, rawY=dataY, fixProcessOrClass="Class", fixed="B", targetProcess=256, targetBenchmarkClass="B", targetFunctionCallNum=-1, csvDirPath="./csv_files")

In [ ]:
resultIs = dictForLatexTable["ft"]
# resultIs
resultIsAtModelBranch = resultIs[resultIs["objectBestModelName"]
                                 == "ModelBranch"]
resultIsAtModelBranchOfNotLowMAPE = resultIsAtModelBranch[
    resultIsAtModelBranch["MAPEOfBestModel"] > 1]
resultIsAtModelBranchOfNotLowMAPE
datumX = resultIsAtModelBranchOfNotLowMAPE["usedDataX"].tolist()
datumY = resultIsAtModelBranchOfNotLowMAPE["usedDataY"].tolist()

datumX
datumY

for dataIndex in range(len(datumX)):
    plt.figure()
    plt.scatter(datumX[dataIndex], datumY[dataIndex])

# returnSeriesOfData(benchmarkName="is", functionName="double_randlc(double_*_double_*)", rawX=dataX, rawY=dataY, fixProcessOrClass="Class", fixed="B", targetProcess=256, targetBenchmarkClass="B", targetFunctionCallNum=-1, csvDirPath="./csv_files")

In [ ]:
# 生データの取得
cgDF = returnCollectedExistingData(benchmarkNames=["cg"], classes=["A", "B", "C", "D"], processes=[
                                   1, 2, 4, 8, 16, 32, 64, 128, 256], csvDirPath="./csv_files/")
cgDF
# ベンチマーククラスがAの情報を取得
cgDFfixedA = cgDF[cgDF["benchmarkClass"] == "A"]
cgDFfixedA
# 関数名のリストを取得
functionNames = sorted(list(set(cgDFfixedA["functionName"])))
print(functionNames)

# 関数名を関数名のリストから抽出
functionNameCG = cgDFfixedA[cgDFfixedA["functionName"] == "CG"]
functionNameCG

# 説明変数と目的変数とをリスト化したものを抽出
# プロセス数
raw_x = functionNameCG['process'].tolist()
# 関数コール回数
raw_y = functionNameCG['functionCallNum'].tolist()

print(f"raw_x={raw_x}")
print(f"raw_y={raw_y}")

bencmarkName = "CG"
functionName = "CG"
fixProcessOrClass = "Class"
fixed = "A"
targetProcess = 256
targetBenchmarkClass = fixed
targetFunctionCallNum = raw_y[-1]
returnSeriesOfData(benchmarkName="benhmarkName", functionName="functionName", rawX=[1, 2, 3], rawY=[
                   1, 2, 3], fixProcessOrClass="Class", fixed="B", targetProcess=256, targetBenchmarkClass="B", targetFunctionCallNum=-1, csvDirPath="./csv_files/")

In [ ]:
%reset

In [ ]:
# ノートブック中で変数のみを記述することでデータフレームをきれいに表示させる設定の有効化
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 問題サイズD, コア数256での関数コール回数を予測する

In [ ]:
# TODO：BT, SP以外のベンチマーク名を入れる
benchmarkNames = ['cg', 'ep', 'ft', 'is', 'lu', 'mg']
classes = ["A", "B", "C", "D"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

dictForSummarizedResult = {}
columnName = ["benchmarkName", "functionName", "score", "relativeErrorRate"]
dfForSummarizedResult = pd.DataFrame(columns=columnName)
for benchmarkName in benchmarkNames:
    # ベンチマークごとにscoreを保持するためのリスト
    listForSummarizedResultPerBenchmarkName = []
    # 学習用生データ
    DF = returnCollectedExistingData(benchmarkNames=[
                                     benchmarkName], classes=classes, processes=processes, csvDirPath="./csv_files/")
    # 重複のない関数名のリスト
    functionNames = list(set(DF["functionName"]))
    usefulFunctionNames = []
    # このループで関数ごとのデータが問題サイズパターン数xコア数パターン数 分だけ存在する関数名のリストを作成する
    for functionName in functionNames:
        # 関数ごとに生データを集計
        dfPerFunction = DF[DF["functionName"] == functionName]
        if len(classes) * len(processes) == len(dfPerFunction):
            usefulFunctionNames.append(functionName)
    if len(usefulFunctionNames) == 0:
        continue
    # 関数ごとのデータを抽出
    for functionName in usefulFunctionNames:
        # 問題サイズを数値化したカラムを追加
        listBenchmarkClass = DF["benchmarkClass"].tolist()
        DFWithNumInBenchmarkClass = DF.assign(
            benchmarkClassInNum=convertBenchmarkClasses_problemSizeInNPB(listBenchmarkClass))
        # 学習用データ
        dfPerFunctionForTrain = DFWithNumInBenchmarkClass[(
            DFWithNumInBenchmarkClass["functionName"] == functionName)]
        dfPerFunctionForTest = DFWithNumInBenchmarkClass[(DFWithNumInBenchmarkClass["functionName"] == functionName) & (
            DFWithNumInBenchmarkClass["benchmarkClass"] == "D") & (DFWithNumInBenchmarkClass["process"] == 256)]

        # x:説明変数, t:目的変数
        trainX = dfPerFunctionForTrain[["process", "benchmarkClassInNum"]]
        trainT = dfPerFunctionForTrain[["functionCallNum"]]
        testX = dfPerFunctionForTest[["process", "benchmarkClassInNum"]]
        testT = dfPerFunctionForTest[["functionCallNum"]]
        # 重回帰分析する
        reg_model = LinearRegression()
        reg_model.fit(trainX, trainT)
        # 関数ごとの結果をベンチマークごとの結果に入れる
        scorePerFunction = reg_model.score(trainX, trainT)
        listForSummarizedResultPerBenchmarkName.append(scorePerFunction)
        # 予測を実施して、相対誤差を算出
        predictedTByTestX = reg_model.predict(testX)
        predictedData = predictedTByTestX[0][0]
        realData = testT["functionCallNum"].tolist()[0]
        relativeErrorPerFunction = abs(predictedData - realData)/realData * 100
        ##
        dfPerFunction = pd.DataFrame(index=columnName, data=[
                                     benchmarkName, functionName, scorePerFunction, relativeErrorPerFunction]).T
        dfForSummarizedResult = dfForSummarizedResult.append(dfPerFunction)

# ( A ~ D ) * (1 ~ 256) のすべての条件を
# 満たしていたら、リストに追加
# 満たしていなければ、なにもしない

In [ ]:
dfForSummarizedResult

In [ ]:
inputDF = dfForSummarizedResult


benchmarkNamesInDF = list(set(dfForSummarizedResult["benchmarkName"].tolist()))

listForLatexTable = []
for benchmarkName in benchmarkNamesInDF:
    print(benchmarkName)
    inputDFPerBenchmark = inputDF[inputDF["benchmarkName"] == benchmarkName]
    meanData = inputDFPerBenchmark.mean()
    print(type(meanData))
    meanData["benchmarkName"] = f"{benchmarkName.upper()}({len(inputDFPerBenchmark)})"
    listForLatexTable.append(meanData)
DF = pd.DataFrame(listForLatexTable)

DF = DF.sort_index(axis='columns')
DF
# relativeErrorの単位は[%]ではない。scoreの値はscore()で取得できたもの
DF.columns = ["ベンチマーク名(関数の個数)", "MAPE(予測対象関数コール回数に対する)", "決定係数"]
print(DF.to_latex(index=False))

In [ ]:
forInputDF = returnDFSummarizedData(
    benchmarkNames=["cg", "ep", "ft", "is", "lu", "mg"],
    classes=["C"],
    processes=[2, 4, 8, 16, 32, 64, 128, 256],
    targetIndex=-1,
    csvDirPath="./csv_files/",
)

benchmarkNames = list(set(forInputDF["benchmarkName"].tolist()))
columnsNames = ["ベンチマーク名(関数の個数)", "MAPE(予測対象関数コール回数に対する)"]
listForRelativeErrorTable = []
for benchmarkName in benchmarkNames:
    forInputDFPerBenchmark = forInputDF[forInputDF["benchmarkName"]
                                        == benchmarkName]
    column1 = f"{benchmarkName.upper()}({len(forInputDFPerBenchmark)})"
    seriesOfMean = forInputDFPerBenchmark.mean()
    seriesOfMeanRelativeErrorRate = seriesOfMean["RelativeErrorRate"]
    column2 = int(seriesOfMeanRelativeErrorRate * 100) / 100
    listForRelativeErrorTable.append([{column1}, {column2}])
print(pd.DataFrame(listForRelativeErrorTable,
      columns=columnsNames).to_latex(index=False))

# forInputDFPerBenchmark

In [ ]:
pd.DataFrame(listForRelativeErrorTable, columns=columnsNames)

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb


benchmarkNames = ['cg', 'ep', 'ft', 'is', 'lu', 'mg']
benchmarkName = 'cg'

classes = ["A", "B", "C", "D"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

# 学習用生データ
DF = returnCollectedExistingData(benchmarkNames=[
                                 benchmarkName], classes=classes, processes=processes, csvDirPath="./csv_files/")
DFByValidFunction = returnDFwithFunctionsExecUnderAllConditions(
    inputDF=DF, classes=classes, processes=processes)
# 問題サイズを数値化したカラムを追加
listBenchmarkClass = DFByValidFunction["benchmarkClass"].tolist()
DFWithNumInBenchmarkClass = DFByValidFunction.assign(
    benchmarkClassInNum=convertBenchmarkClasses_problemSizeInNPB(listBenchmarkClass))
validFunctionNames = list(
    set(DFWithNumInBenchmarkClass["functionName"].tolist()))
for validFunctionName in validFunctionNames:
    pass

# 学習用データを抽出
# 予測対象データを抽出

classesTarget = classes[-1]
processesTarget = processes[-1]

In [ ]:
DFWithNumInBenchmarkClass

In [13]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# class Models
# 各モデルのオブジェクトデータを保持している。
# 学習用データ・予測対象データを保持している
# 引数名とその説明
# inputDF：入力データの全てを保持したDF（説明変数・目的変数・ベンチマーク名・関数名を最低限保持している）
# expVarColNames：inputDFの列名の中で、説明変数として用いるカラム名のリスト
# resVarColNames：inputDFの列名の中で、説明変数として用いるカラム名のリスト
# targetDF：inputDFとデータ構成は同じだが、予測対象のデータがセットされている
# modelNames：実施するモデル名を指定できる(["modelLin", "modelIp", "modelLog"])

class Models:

    def __init__(self, inputDF, expVarColNames, resVarColNames, targetDF=None, modelNames=["modelLin", "modelIp", "modelLog"]):
        self.inputDF = inputDF
        self.expVarColNames = expVarColNames
        if len(resVarColNames) > 1:
            warnings.warn("説明変数が複数個存在しています")
        self.resVarColNames = resVarColNames
        self.targetDF = targetDF
        self.functionName = ""
        self.benchmarkName = ""
        self.modelNames = modelNames

        if ("modelLin" in self.modelNames):
            self.objectModelLin = ModelLinForMultipleRegression(inputDF, explanatoryVariableColumnNames=expVarColNames, responseVariableColumnNames=resVarColNames, targetDF=targetDF)
        if ("modelIp" in self.modelNames):
            self.objectModelIp = ModelIpForMultipleRegression(inputDF, explanatoryVariableColumnNames=expVarColNames, responseVariableColumnNames=resVarColNames, targetDF=targetDF)
        if ("modelLog" in self.modelNames):
            self.objectModelLog = ModelLogForMultipleRegression(inputDF, explanatoryVariableColumnNames=expVarColNames, responseVariableColumnNames=resVarColNames, targetDF=targetDF)


    def setUpDataBeforeCalcLr(self):
        if ("modelLin" in self.modelNames):
            self.objectModelLin.setUpDataBeforeCalcLr()
        if ("modelIp" in self.modelNames):
            self.objectModelIp.setUpDataBeforeCalcLr()
        if ("modelLog" in self.modelNames):
            self.objectModelLog.setUpDataBeforeCalcLr()

    def calcLr(self):
        if ("modelLin" in self.modelNames):
            self.objectModelLin.calcLr()
        if ("modelIp" in self.modelNames):
            self.objectModelIp.calcLr()
        if ("modelLog" in self.modelNames):
            self.objectModelLog.calcLr()

    # inputDF：__init__()でのinputDFとDF構成は同じ
    def predict(self, inputDF):
        pass

    # 学習用データへの適合度（MAPE[%]）を計算する
    def calcMAPE(self):
        # MAPEatTrain:辞書
        # キーはmodelNamesの要素、バリューは学習データの適合度としてのMAPE
        MAPEatTrain = {}
        if len(self.resVarColNames) > 1:
            warnings.warn("説明変数が複数カラムに及んでいるため、正常な動作を期待できません")
        realData = self.inputDF[self.resVarColNames[0]].tolist()
        if ("modelLin" in self.modelNames):
            predictedDataAtLin = self.objectModelLin.predict(self.inputDF[self.expVarColNames])
            modelLinMAPEatTrain = returnMapeScore(realData, predictedDataAtLin)
            MAPEatTrain["modelLin"] = modelLinMAPEatTrain
        if ("modelIp" in self.modelNames):
            predictedDataAtIp  = self.objectModelIp.predict(self.inputDF[self.expVarColNames])
            modelIpMAPEatTrain = returnMapeScore(realData, predictedDataAtIp)
            MAPEatTrain["modelIp"] = modelIpMAPEatTrain
        if ("modelLog" in self.modelNames):
            predictedDataAtLog = self.objectModelLog.predict(self.inputDF[self.expVarColNames])
            modelLogMAPEatTrain = returnMapeScore(realData, predictedDataAtLog)
            MAPEatTrain["modelLog"] = modelLogMAPEatTrain
        self.MAPEatTrain = MAPEatTrain
        
    # calcMAPEで計算した辞書を返す関数
    # 返す辞書がない場合は空の辞書を返す
    def returnCalculatedMAPE(self):
        if self.MAPEatTrain is None:
            reutnrn({})
        else:
            return(self.MAPEatTrain)

    # 予測対象データとの相対誤差率を計算する
    # 引数 targetDF:本オブジェクト構築時に必要になるinputDFをデータ構造が同じDF
    def calcRelativeErrorRate(self, targetDF=None):
        # relativeErrorRateDict:辞書
        # キーはmodelNamesの要素、バリューは絶対相対誤差率
        relativeErrorRateDict = {}
        # （すでに予測対象の説明変数データがある or targetDF is not None）なら問題ない。
        if ((self.targetDF is None) and (targetDF is None)):
            warnings.warn("相対誤差率を計算するための真値が与えられていません。")
            return (-1)
        
        if len(self.resVarColNames) == 0:
            warnings.warn("説明変数のカラム名が複数設定されています")
        if targetDF is None :
            if len(self.targetDF) > 1:
                warnings.warn("ターゲットとなるDFに要素が2つ以上含まれています。")
            realData = self.targetDF[self.resVarColNames[0]]
        else:
            if len(targetDF) > 1:
                warnings.warn("ターゲットとなるDFに要素が2つ以上含まれています。")
            realData = targetDF[self.resVarColNames[0]]
        
        # realData は DataFrame なので、それをリスト化して、最初の要素のみ保持する
        realData = realData.tolist()[0]
    
        if ("modelLin" in self.modelNames):
            predictedData = self.objectModelLin.predict(targetDF[self.expVarColNames]).tolist()[0][0]
            print(f"type(realData)={type(realData)}, type(predictedData)={type(predictedData)}")
            print(f"realData={realData}")
            print(f"predictedData={predictedData}")
            relativeErrorRateDict["modelLin"] = returnRelativeErrorRate(realNum=realData, predictedNum=predictedData)
        if ("modelIp" in self.modelNames):
            predictedData = self.objectModelIp.predict(targetDF[self.expVarColNames])
            relativeErrorRateDict["modelIp"] = returnRelativeErrorRate(realNum=realData, predictedNum=predictedData)
        if ("modelLog" in self.modelNames):
            predictedData = self.objectModelLog.predict(targetDF[self.expVarColNames])
            relativeErrorRateDict["modelLog"] = returnRelativeErrorRate(realNum=realData, predictedNum=predictedData)
        self.relativeErrorRateDict = relativeErrorRateDict

    # calcRelativeErrorRate()で計算した辞書を返す関数
    # 返す辞書がない場合は空の辞書を返す
    def returnRelativeErrorRateDict(self):
        if self.relativeErrorRateDict is None:
            return ({})
        else:
            return(self.relativeErrorRateDict)

    # 関数名・ベンチマーク名を更新する
    def updateFunctionAndBenchmarkName(self, functionName=None, benchmarkName=None):
        # TODO:各種引数が空の場合は更新しない
        if functionName is not None:
            self.functionName = functionName
        if benchmarkName is not None:
            self.benchmarkName = benchmarkName

    def returnFunctionName(self):
        return(self.functionName)
    def returnBenchmarkName(self):
        return(self.benchmarkName)

def test_Models():
    # inputDFを準備
    # 説明変数
    plotX = np.linspace(1, 20, 10)
    plotY = np.linspace(21, 40, 10)
    plotZ = np.linspace(41, 60, 10)
    # 目的変数
    plotTforLin = plotX + 2*plotY + 3*plotZ + 4
    plotTforIp  = 1/plotX + 2/plotY + 3/plotZ + 4
    plotTforLog = np.log10(plotX) + 2*np.log10(plotY) + 3*np.log10(plotZ) + 4
    inputDF = pd.DataFrame({"plotX":plotX, "plotY":plotY, "plotZ":plotZ, "plotTforLin":plotTforLin, "plotTforIp":plotTforIp, "plotTforLog":plotTforLog})
    
    # functionNameを準備
    functionName = "functionName"
    # benchmarkNameを準備
    benchmarkName = "benchmarkName"
    
    inputDF["functionNames"] = functionName
    inputDF[benchmarkName] = benchmarkName
    # targetDFを準備
    targetDF = inputDF.tail(1)
    
    columnNames = inputDF.columns.tolist()
    # expVarColNamesを準備
    expVarColNames = columnNames[:3]
    # resVarColNamesを準備
    resVarColNames = columnNames[-3:]
    resVarColNamesForLin = ["plotTforLin"]
    resVarColNamesForIp  = ["plotTforIp"]
    resVarColNamesForLog = ["plotTforLog"]

    # インスタンスを作成
    modelsLin = Models(inputDF=inputDF, expVarColNames=expVarColNames, resVarColNames=resVarColNamesForLin, targetDF=targetDF)
    modelsIp  = Models(inputDF=inputDF, expVarColNames=expVarColNames, resVarColNames=resVarColNamesForIp , targetDF=targetDF)
    modelsLog = Models(inputDF=inputDF, expVarColNames=expVarColNames, resVarColNames=resVarColNamesForLog, targetDF=targetDF)
    # 予測に必要な初期化作業を開始
    modelsLin.setUpDataBeforeCalcLr()
    modelsIp.setUpDataBeforeCalcLr()
    modelsLog.setUpDataBeforeCalcLr()
    # モデル構築を実施
    modelsLin.calcLr()
    modelsIp.calcLr()
    modelsLog.calcLr()
    # TODO:予測をして学習データに対するMAPEを計算し、その値がそれぞれ小さいことを確認
    modelsLin.calcMAPE()
    dictCalcedMAPEatLin = modelsLin.returnCalculatedMAPE()
    modelsIp.calcMAPE()
    dictCalcedMAPEatIp = modelsIp.returnCalculatedMAPE()
    modelsLog.calcMAPE()
    dictCalcedMAPEatLog = modelsLog.returnCalculatedMAPE()
    print(dictCalcedMAPEatLin)
    print(dictCalcedMAPEatIp)
    print(dictCalcedMAPEatLog)
    # TODO:相対誤差率を計算し、それが小さいことを確認
    modelsLin.calcRelativeErrorRate(targetDF=targetDF)
    modelsIp.calcRelativeErrorRate(targetDF=targetDF)
    modelsLog.calcRelativeErrorRate(targetDF=targetDF)
    relativeErrorRateDictAtLin = modelsLin.returnRelativeErrorRateDict()
    relativeErrorRateDictAtIp  = modelsIp.returnRelativeErrorRateDict()
    relativeErrorRateDictAtLog = modelsLog.returnRelativeErrorRateDict()
    print(f"relativeErrorRateDictAtLin={relativeErrorRateDictAtLin}")
    print(f"relativeErrorRateDictAtIp ={relativeErrorRateDictAtIp}")
    print(f"relativeErrorRateDictAtLog={relativeErrorRateDictAtLog}")
    # 関数名・ベンチマーク名を更新する関数のテスト
    # updateFunctionAndBenchmarkName()
    print(f"modelsLin.returnFunctionName()={modelsLin.returnFunctionName()}, modelsIp.returnFunctionName()={modelsIp.returnFunctionName()}, modelsLog.returnFunctionName()={modelsLog.returnFunctionName()}")
    print(f"modelsLin.returnBenchmarkName()={modelsLin.returnBenchmarkName()}, modelsIp.returnBenchmarkName()={modelsIp.returnBenchmarkName()}, modelsLog.returnBenchmarkName()={modelsLog.returnBenchmarkName()}")
    modelsLin.updateFunctionAndBenchmarkName(functionName=functionName)
    modelsIp.updateFunctionAndBenchmarkName(functionName=functionName)
    modelsLog.updateFunctionAndBenchmarkName(functionName=functionName)
    print(f"modelsLin.returnFunctionName()={modelsLin.returnFunctionName()}, modelsIp.returnFunctionName()={modelsIp.returnFunctionName()}, modelsLog.returnFunctionName()={modelsLog.returnFunctionName()}")
    print(f"modelsLin.returnBenchmarkName()={modelsLin.returnBenchmarkName()}, modelsIp.returnBenchmarkName()={modelsIp.returnBenchmarkName()}, modelsLog.returnBenchmarkName()={modelsLog.returnBenchmarkName()}")
    modelsLin.updateFunctionAndBenchmarkName(benchmarkName=benchmarkName)
    modelsIp.updateFunctionAndBenchmarkName(benchmarkName=benchmarkName)
    modelsLog.updateFunctionAndBenchmarkName(benchmarkName=benchmarkName)    
    print(f"modelsLin.returnFunctionName()={modelsLin.returnFunctionName()}, modelsIp.returnFunctionName()={modelsIp.returnFunctionName()}, modelsLog.returnFunctionName()={modelsLog.returnFunctionName()}")
    print(f"modelsLin.returnBenchmarkName()={modelsLin.returnBenchmarkName()}, modelsIp.returnBenchmarkName()={modelsIp.returnBenchmarkName()}, modelsLog.returnBenchmarkName()={modelsLog.returnBenchmarkName()}")
    # assertする

    pass

test_Models()

{'modelLin': array([1.46970337e-14]), 'modelIp': array([0.14415395]), 'modelLog': array([0.03990015])}
{'modelLin': array([3.35330871]), 'modelIp': array([1.25003187e-14]), 'modelLog': array([0.21768522])}
{'modelLin': array([1.07618955]), 'modelIp': array([0.03316237]), 'modelLog': array([2.3043592e-14])}
type(realData)=<class 'float'>, type(predictedData)=<class 'float'>
realData=284.0
predictedData=284.0
type(realData)=<class 'float'>, type(predictedData)=<class 'float'>
realData=4.15
predictedData=3.9965653407923334
type(realData)=<class 'float'>, type(predictedData)=<class 'float'>
realData=13.839603729470838
predictedData=14.030088439648038
relativeErrorRateDictAtLin={'modelLin': 0.0, 'modelIp': array([[0.196]]), 'modelLog': array([[0.05]])}
relativeErrorRateDictAtIp ={'modelLin': 3.697, 'modelIp': array([[0.]]), 'modelLog': array([[0.269]])}
relativeErrorRateDictAtLog={'modelLin': 1.376, 'modelIp': array([[0.044]]), 'modelLog': array([[0.]])}
modelsLin.returnFunctionName()=, mod